#Import the newest posts and store data in a separate file

In [20]:
import requests
import pandas as pd
import json

with open('appid.txt', 'r') as f:
    CLIENT_ID = f.readline().strip()
    SECRET_KEY = f.readline().strip()

auth = requests.auth.HTTPBasicAuth(CLIENT_ID, SECRET_KEY)

with open('pw.txt', 'r') as f:
    username = f.readline().strip()
    pw = f.readline().strip()
    
data = {
    'grant_type' : 'password',
    'username' : username,
    'password' : pw
}
headers = {'User-Agent': 'MyAPI/0.0.1'}

res = requests.post('https://www.reddit.com/api/v1/access_token',
                    auth=auth, data=data, headers=headers)
TOKEN = res.json()['access_token']

headers['Authorization'] = f'bearer {TOKEN}'

# Obtain 100 of the newest posts 
params = {
    'limit': 100
    }
res = requests.get('http://oauth.reddit.com/r/umd/new/',
                 headers = headers, params=params)
df = pd.DataFrame()

for post in res.json()['data']['children']:
    df = df._append({
        'title': post['data']['title'],
        'upvotes': post['data']['ups'],
        'downvotes':post['data']['downs'],
        'comment_count': post['data']['num_comments'],
        'flair': post['data']['link_flair_text']
        
    }, ignore_index=True)

df.to_csv(path_or_buf='umdtoppostsdata.csv', mode = 'w', index = False)

Read Reddit informaiton from data file and use to predict whether or not post will be more or less controversial 

In [ ]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
    
subreddit_data = pd.read_csv('umdtoppostsdata.csv')
subreddit_data['controversial'] = 0

# Set X to the original data and Y to a post that I determine 
# to be controversial

# Posts tend to be controversial when they have a similar number of upvotes
# and downvotes and a large number of engagement (comments)

# A post can also be controversial if there are more comments than upvotes


def is_controversial(row):
    
    # If the upvote and downvote count are similar, the post is more controversial
    
    controverisal = 0
    
    upvotes = row['upvotes']
    downvotes = row['downvotes']
    comment_num = row['comment_count']
    
    # If there are more downvotes than upvotes, the post is immediately controversial
    
    if (downvotes > 0):
        controverisal = 1
    
    # If the number of comments is substantially bigger than the upvote ratio,
    # The post is likely more controversial
    if (not controverisal):
        if (comment_num >= 1.5 * (upvotes)):
            controverisal = 1
        if (comment_num >= upvotes / 2):
            controverisal = 1
    return controverisal

subreddit_data['controversial'] = subreddit_data.apply(is_controversial, 1)
X = subreddit_data[['title', 'upvotes', 'downvotes', 'flair']]
Y = subreddit_data[['controversial']]

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)





,controversial
0,1
1,0
2,1
3,1
4,1
...,...
95,0
96,0
97,0
98,0
